#  卷积神经网络+selu

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
print(tf.__version__)
print(sys.version_info)
for module in np, pd ,sklearn, tf, keras:
    print(module.__name__,module.__version__)

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(
        x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28,1)
x_valid_scaled = scaler.transform(
        x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28,1)
x_test_scaled = scaler.transform(
        x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28,1)

In [ ]:
# kernel_size卷积窗口的长度
# pool_size池化窗口的长度
# 池化操作通常不重叠，不补0.pool_size=步长，用于减小图像尺寸，减少计算量
# filter输出的通道，即卷积核的数量
# padding='same'指填充padding使得输入输出大小相同，默认‘valid’指不填充
# strides=(1, 1)卷积时在每一维度上的步长
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',
                        input_shape =[28,28,1], activation='selu'),
    keras.layers.Conv2D(filters = 32,kernel_size=3,padding='same',
                        activation='selu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',
                         activation='selu'),
    keras.layers.Conv2D(filters = 64,kernel_size=3,padding='same',
                        activation='selu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',
                        activation='selu'),
    keras.layers.Conv2D(filters = 128,kernel_size=3,padding='same',
                        activation='selu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),#连接全连接层前要flatten展平
    keras.layers.Dense(128, activation= 'selu'),
    keras.layers.Dense(10,activation='softmax')
])    

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             optimizer = 'adam',
             metrics = ['accuracy'])


In [ ]:
model.layers#查看模型层数

In [ ]:
model.summary()

In [ ]:
logdir = 'data/cnn-selu-callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,"fashion_mnist_model.h5")
callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,
                                               save_best_only = True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)
]
history = model.fit(x_train_scaled,y_train,epochs=10,
                          validation_data=(x_valid_scaled,y_valid),
                          callbacks = callbacks)

In [ ]:
history.history

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=[8,5])
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()

plot_learning_curves(history)